# Buckling under lateral force: ABAQUS sims and plots

In [ ]:
import bikewheellib as bl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from doetools import LateralBucklingDOE, wheel_library
from IPython.display import display
from shutil import copy2
from imp import find_module

%matplotlib inline

## ABAQUS Explicit dynamic

In [ ]:
exp_dir = '../data/doe/doe_lat_buckling_exp'

if False:

    doe_opts = {'spk_paired': False, 'spk_eltype': 'beam',
                'sim_type': 'exp', 'sim_u3': 0.04, 'sim_alpha': 1e3}

    doe = LateralBucklingDOE(out_dir=exp_dir)

    # Create wheel
    wheel = wheel_library.create_predefined_wheel('vintage_rd')

    # Get buckling tension
    Tc_theor, nc = bl.calc_buckling_tension(wheel)

    for Tn in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
        jobname = 'Tn{:.2f}'.format(Tn)

        wheel = wheel_library.create_predefined_wheel('vintage_rd')
        
        doe_opts.update({'jobname': jobname, 'spk_Tn': Tn, 'spk_T': Tn * Tc_theor})
        doe.add_experiment(wheel, opts=doe_opts)

    print('\nCreated {0:d} simulations'.format(len(doe.db)))

    doe.write_input_files(N_batches=3)
    doe.to_csv()
    
    # Copy postprocessing script
    copy2(src=find_module('doetools')[1] + '/postproc_lat_buckling.py',
          dst=exp_dir)

## ABAQUS Standard Riks

In [ ]:
riks_dir = '../data/doe/doe_lat_buckling_riks'

if True:

    doe_opts = {'spk_paired': False, 'spk_eltype': 'beam',
                'sim_type': 'riks', 'sim_u3': 0.06, 'sim_Pmax': 600.}

    doe = LateralBucklingDOE(out_dir=riks_dir)

    # Create wheel
    wheel = wheel_library.create_predefined_wheel('vintage_rd')

    # Get buckling tension
    Tc_theor, nc = bl.calc_buckling_tension(wheel)

    for Tn in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
        jobname = 'Tn{:.2f}'.format(Tn)
        
        wheel = wheel_library.create_predefined_wheel('vintage_rd')

        doe_opts.update({'jobname': jobname, 'spk_Tn': Tn,'spk_T': Tn*Tc_theor})
        doe.add_experiment(wheel, opts=doe_opts)

    print('\nCreated {0:d} simulations'.format(len(doe.db)))

    doe.write_input_files(N_batches=4)
    doe.to_csv()
    
    # Copy postprocessing script
    copy2(src=find_module('doetools')[1] + '/postproc_lat_riks.py',
          dst=riks_dir)

## Plots

In [ ]:
# Load DOE database
print 'Loading database...'
doe_r = LateralBucklingDOE(out_dir=riks_dir, db_file=riks_dir+'/_doe_db.csv')

# Populate DOE database with results
print 'Extracting results...'
doe_r.extract_results()
print '\nDone'

In [ ]:
cp = sns.color_palette('Reds', 6)

with plt.style.context('seaborn-paper'):

    fig, ax = plt.subplots(figsize=(3, 3))

    j = 0
    for i, r in doe.db[doe.db['spk_Tn'].between(0.1, 0.6)].iterrows():
        pd_data = pd.read_csv(riks_dir + '/' + r.name + '_riks_Pd.csv')

        u = pd_data['U3 [m]'] - pd_data['U3 [m]'][0]
        P = pd_data['RF3 [N]']

        ax.plot(pd_data['U3 [m]'] - pd_data['U3 [m]'][0], pd_data['RF3 [N]'],
                color=cp[j],
                label='{:.1f}'.format(r.spk_Tn))
        j += 1

        i_zero = np.where(np.diff(np.sign(P)))[0][1:]

        ax.plot(u[i_zero], P[i_zero], 'k*', label='_nolegend_', markersize=8)

    ax.set_xlim([0., 0.06])
    ax.set_xlabel('Lateral displacement [mm]')
    ax.set_ylabel('Lateral load [N]')

    ax.set_ylim([-200., 600.])

    plt.tight_layout()
    # plt.savefig('../figs/buckling_ext_loads/_python_lat_riks.pdf')